In [1]:
import matplotlib.pyplot as plt
import sys
import logging
import os
from cellpose import models, train
from sklearn.model_selection import train_test_split
import napari
from magicgui import magicgui
import numpy as np
from tifffile import imwrite

src_path = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(src_path)
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('napari').setLevel(logging.WARNING)
logging.getLogger('in_n_out').setLevel(logging.WARNING)
logging.getLogger('numcodecs').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)

from AngelFISH.src import Receipt, load_data
from AngelFISH.src.Steps import download_data, return_data

INFO:luigi-interface:Loaded []
2025-08-25 11:22:10 |  WARNING | Loading postgres module without psycopg2 nor pg8000 installed. Will crash at runtime if postgres functionality is used.
2025-08-25 11:22:10 |  WARNING | Loading S3 module without the python package boto3. Will crash at runtime if S3 functionality is used.


In [2]:
receipt = Receipt(
    analysis_name = 'cellpose_segmentation',
    nas_location = r'Users\Jack\_ImageDatasets\IntronDiffusion\08072025_TriptolideTimeSeries_JF001\JF001_15min_5uM_3',
    local_location = None,
    data_loader = 'pycromanager_data_loader'
    )

In [3]:
receipt = download_data(receipt, 'download_data').process()

local file already exist


In [4]:
# Parameters
mask_name = 'handseg_MCPGFP'
channel_of_interest = 1
model_name = 'MCPGFP_nuclei'
do_3d = False # TODO This doesnt work

In [5]:
data = load_data(receipt)
images = data['images'][:,:,channel_of_interest,:,:,:]
pp, tt, zz, yy, xx = images.shape 
if not do_3d:
    images = np.max(images, axis=2)

# Start viewer
viewer = napari.Viewer()

# Add image and labels
viewer.add_image(images, name="image")

# Run Napari
napari.run()



Dataset opened                


In [7]:
# After closing viewer, retrieve labels array
final_labels = viewer.layers['Labels'].data

if not do_3d and zz > 1: # expand 2d image to 3d 
    new_masks = np.zeros((pp, tt, zz, yy, xx))
    for p in range(pp):
        for t in range(tt):
            for z in range(zz):
                new_masks[p, t, z, :, :] = np.max(final_labels[p, t], axis=0)
    final_labels = new_masks

imwrite(os.path.join(receipt['dirs']['masks_dir'], f'{mask_name}.tif'), final_labels)

print("Labels shape:", final_labels.shape)
print("Unique labels:", np.unique(final_labels))

Labels shape: (64, 1, 21, 512, 512)
Unique labels: [0. 1. 2. 3. 4. 5. 6.]


In [8]:
return_data(receipt, 'return_data').process()

INFO:SMB.SMBConnection:Authentication with remote machine "MUNSKY-NAS.ENGR.COLOSTATE.EDU" for user "formanj" will be using NTLM v2 authentication (with extended security)
INFO:SMB.SMBConnection:Now switching over to SMB2 protocol communication
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_NEGOTIATE" (command:0x0000 flags:0x0001)
INFO:SMB.SMBConnection:SMB2 dialect negotiation successful
DEBUG:SMB.SMBConnection:Received SMB2 message "SMB2_COM_SESSION_SETUP" (command:0x0001 flags:0x0001)
INFO:SMB.SMBConnection:Performing NTLMv2 authentication (on SMB2) with server challenge "b'8a7deb5f716d3a88'"
INFO:SMB.SMBConnection:Performing NTLMv2 authentication (on SMB2) with server challenge "b'8a7deb5f716d3a88'"
DEBUG:SMB.SMBConnection:NT challenge response is "b'5af4a44d0ab1da9f03ece4fc89d3599e01010000000000000000000000000000b8c54507821e5736000000000200100045004e00470052005f0044004f004d00010014004d0055004e0053004b0059002d004e00410053000400240065006e00670072002e0063006f006c006f007300740

KeyboardInterrupt: 

In [13]:
# load data
data = load_data(receipt)
images = data['images'][:,:,channel_of_interest,:,:,:]
labels = data[mask_name]

# image_list = [np.expand_dims(np.asarray(images[p, t]), axis=-1) for p in range(images.shape[0]) for t in range(images.shape[1])]
if do_3d:
    image_list = [np.moveaxis(np.asarray(images[p, t]), 0, -1) for p in range(images.shape[0]) for t in range(images.shape[1])]
    label_list = [np.moveaxis(np.asarray(labels[p, t]), 0, -1) for p in range(labels.shape[0]) for t in range(labels.shape[1])]
else:
    image_list = [np.max(np.asarray(images[p, t]),axis=0) for p in range(images.shape[0]) for t in range(images.shape[1])]
    label_list = [np.max(np.asarray(labels[p, t]),axis=0) for p in range(labels.shape[0]) for t in range(labels.shape[1])]

image_list = [im[np.newaxis, ...] for im in image_list]
# label_list = [lbl[np.newaxis, ...] for lbl in label_list]

# find index with masks
good_index = [i for i, mask in enumerate(label_list) if not np.all(mask == 0)]
image_list = [image_list[i] for i in good_index]
label_list = [label_list[i] for i in good_index]

# split data
training_images, testing_images, training_labels, testing_labels = train_test_split(
    image_list, label_list, test_size=0.2, random_state=42
)

# assert np.all(im.shape == lbl.shape for im, lbl in zip(training_images, training_labels))
# assert np.all(im.shape == lbl.shape for im, lbl in zip(testing_images, testing_labels))

# train new model
model = models.CellposeModel(gpu=True, model_type='livecell')

path = train.train_seg(model.net,
                    train_data=training_images, 
                    train_labels=training_labels,
                    test_data=testing_images, 
                    test_labels=testing_labels,
                    channel_axis=0,
                    weight_decay=0.1, 
                    learning_rate=1e-5,
                    n_epochs=180,
                    model_name=model_name, 
                    min_train_masks=1
                    )



Dataset opened                


c:\Users\Jack\Documents\GitHub\AngelFISH\.venv\Lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU (CUDA)
INFO:cellpose.models:>>>> no model weights loaded
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 15/15 [00:03<00:00,  3.94it/s]
INFO:cellpose.dynamics:computing flows for labels
100%|██████████| 4/4 [00:00<00:00,  4.32it/s]
INFO:cellpose.train:>>> computing diameters
100%|██████████| 4/4 [00:00<00:00, 323.65it/s]
INFO:cellpose.train:>>> normalizing {'lowhigh': None, 'percentile': None, 'normalize': True, 'norm3D': False, 'sharpen_radius': 0, 'smooth_radius': 0, 'tile_norm_blocksize': 0, 'tile_norm_smooth3D': 1, 'invert': False}
INFO:cellpose.train:>>> n_epochs=180, n_train=15, n_test=4
INFO:cellpose.train:>>> AdamW, learning_rate=0.00001, weight_decay=

In [14]:
path

WindowsPath('c:/Users/Jack/Documents/GitHub/AngelFISH/models/models/MCPGFP_nuclei')

In [ ]:
# train_losses = model.net.train_losses
# test_losses = model.net.test_losses

# # display losses
# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Testing Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.title('Training and Testing Losses')
# plt.show()

In [ ]:
first_false_index = next(i for i, lbl in enumerate(label_list) if not np.any(lbl > 0))
print("First index where np.any(label_list[i] > 0) is False:", first_false_index)

In [ ]:
plt.imshow(training_images[0])
plt.show()

In [ ]:
training_images[0].shape

In [ ]:
training_labels[0].shape

In [ ]:
model.net